# Homework 2
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories.

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work.

In [ ]:
import platform
display(platform.system())
import os
file_download_link = 'https://www.dropbox.com/scl/fi/j1dxtqjerbdbl81e05nvp/hw4data.zip?rlkey=8mkxz4j8lngziok6on782a8u4&dl=0'
if os.name == 'nt':
    print('Please download your dataset here:', file_download_link)
else:
    # We need to first download the data here:
    !wget -O data.zip "$file_download_link" -o /dev/null
    !unzip data.zip > /dev/null

'Linux'

In [ ]:
!sed 's/,/\t/g' garments_worker_productivity.csv > garments_worker_productivity.tsv

In [ ]:
# If your data is on google drive then uncomment the code below to access
# your google drive.
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 10.7 MB/s eta 0:00:00


# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`.

Realize that `quarter` is not referring to certain months of the year, but certain days of the month. "Quarter1" represents days 1-7, "Quarter2" days 8-14, "Quarter3" days 15-21, "Quarter4" days 22-28, and "Quarter5" days 29-31

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - You must use `df.describe()` or `df.info()` to display the data after preprocessing. **No credit** will be given if this step is omitted.
 - Fill the empty values with 0.


In [ ]:
import pandas as pd
import numpy as np
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('./garments_worker_productivity.csv')
# Drop the column date
df = df.drop(columns=['date'], axis = 1)

In [ ]:
# Print out all the unique elements for categorical attributes
categorical_attributes = ['day', 'quarter', 'department', 'team']
for attr in categorical_attributes:
    print(f"Unique values in {attr}:")
    print(df[attr].unique())
    print()

Unique values in day:
['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']

Unique values in quarter:
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']

Unique values in department:
['sweing' 'finishing ' 'finishing']

Unique values in team:
[ 8  1 11 12  6  7  2  3  9 10  5  4]



In [ ]:
# Remap duplicated items in categorical attributes
def clean_categorical_data(x):
    return str(x).strip().lower()
    for attr in categorical_attributes:
        df[attr] = df[attr].apply(clean_categorical_data)

In [ ]:
# Print unique values again to check for any remaining duplicates
for attr in categorical_attributes:
    print(f"Unique values in {attr} after cleaning:")
    print(df[attr].unique())
    print()

Unique values in day after cleaning:
['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']

Unique values in quarter after cleaning:
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']

Unique values in department after cleaning:
['sweing' 'finishing ' 'finishing']

Unique values in team after cleaning:
[ 8  1 11 12  6  7  2  3  9 10  5  4]



In [ ]:
# Create 'satisfied' column
df['satisfied'] = (df['actual_productivity'] >= df['targeted_productivity']).astype(int)

In [ ]:
# Drop 'actural_productivity' and 'targeted_productivity' columns
df = df.drop(columns=['actual_productivity', 'targeted_productivity'], axis = 1)

In [ ]:
# Find and print out columns with empty values
empty_columns = df.columns[df.isna().any()].tolist()
print(empty_columns)

['wip']


In [ ]:
# Fill empty values with 0
df = df.fillna(0)

In [ ]:
# Verify that there are no more null values
print("\nRemaining null values:")
print(df.isnull().sum())


Remaining null values:
quarter               0
department            0
day                   0
team                  0
smv                   0
wip                   0
over_time             0
incentive             0
idle_time             0
idle_men              0
no_of_style_change    0
no_of_workers         0
satisfied             0
dtype: int64


In [ ]:
# Display the first few rows of the processed dataframe
print(df.head())

    quarter  department       day  team    smv     wip  over_time  incentive  \
0  Quarter1      sweing  Thursday     8  26.16  1108.0       7080         98   
1  Quarter1  finishing   Thursday     1   3.94     0.0        960          0   
2  Quarter1      sweing  Thursday    11  11.41   968.0       3660         50   
3  Quarter1      sweing  Thursday    12  11.41   968.0       3660         50   
4  Quarter1      sweing  Thursday     6  25.90  1170.0       1920         50   

   idle_time  idle_men  no_of_style_change  no_of_workers  satisfied  
0        0.0         0                   0           59.0          1  
1        0.0         0                   0            8.0          1  
2        0.0         0                   0           30.5          1  
3        0.0         0                   0           30.5          1  
4        0.0         0                   0           56.0          1  


In [ ]:
# Save the processed dataframe to a new CSV file
df.to_csv('processed_garments_worker_productivity_processed.csv', index=False)
print("\nProcessed data saved to 'processed_garments_worker_productivity_processed.csv'")


Processed data saved to 'processed_garments_worker_productivity_processed.csv'


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attributes**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.
 - You **must** show the first five row of your encoded dataset, as well as the shape of your train test split. **No credit** will be given if this step is omitted.

In [ ]:
# Remember to continue the task with your processed data from Exercise 1
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('processed_garments_worker_productivity_processed.csv')

In [ ]:
# Encode categorical variables
categorical_attributes = ['day', 'quarter', 'department', 'team']
le = LabelEncoder()

In [ ]:
for attr in categorical_attributes:
    df[attr] = le.fit_transform(df[attr])
    print(f"Encoded values for {attr}:")
    print(dict(zip(le.classes_, le.transform(le.classes_))))
    print()

Encoded values for day:
{'Monday': 0, 'Saturday': 1, 'Sunday': 2, 'Thursday': 3, 'Tuesday': 4, 'Wednesday': 5}

Encoded values for quarter:
{'Quarter1': 0, 'Quarter2': 1, 'Quarter3': 2, 'Quarter4': 3, 'Quarter5': 4}

Encoded values for department:
{'finishing': 0, 'finishing ': 1, 'sweing': 2}

Encoded values for team:
{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11}



In [ ]:
# Split the data into features (X) and target variable (y)
X = df.drop(columns=['satisfied'], axis=1)
y = df['satisfied']

In [ ]:
# Split the data into training and testing sets (80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Shape of training set:", X_train.shape)
print("Shape of testing set:", X_test.shape)

Shape of training set: (957, 12)
Shape of testing set: (240, 12)


In [ ]:
# Display the first few rows of the encoded features
print(X_train.head())

      quarter  department  day  team    smv    wip  over_time  incentive  \
1189        1           2    5     7  30.48  914.0       6840         30   
575         0           1    0     0   3.94    0.0       2280          0   
76          0           1    0     9   2.90    0.0        960          0   
731         1           0    3     3   4.15    0.0       1800          0   
138         1           2    3    11  11.61  548.0      15120         63   

      idle_time  idle_men  no_of_style_change  no_of_workers  
1189        0.0         0                   1           57.0  
575         0.0         0                   0           19.0  
76          0.0         0                   0            8.0  
731         0.0         0                   0           15.0  
138         0.0         0                   0           31.5  


In [ ]:
# Display the first few rows of the target variable
print(y_train.head())

1189    1
575     0
76      1
731     1
138     1
Name: satisfied, dtype: int64


### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [ ]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Select only categorical attributes
categorical_attributes = ['day', 'quarter', 'department', 'team']
X_train_cat = X_train[categorical_attributes]
X_test_cat = X_test[categorical_attributes]

In [ ]:
# Create and train the Categorical Naïve Bayes classifier
cnb = CategoricalNB()
cnb.fit(X_train_cat, y_train)

CategoricalNB()

In [ ]:
# Make predictions on the test set
y_pred = cnb.predict(X_test_cat)

In [ ]:
# Generate and print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.19      0.27        63
           1       0.76      0.93      0.84       177

    accuracy                           0.73       240
   macro avg       0.62      0.56      0.55       240
weighted avg       0.69      0.73      0.69       240



In [ ]:
# Print the accuracy score
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Accuracy: 0.7333


### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [ ]:
# Remember to do this task with your processed data from Exercise 2.1
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

In [ ]:
# Select only numerical attributes
numerical_attrs = ['no_of_workers', 'no_of_style_change', 'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men']
X_train_num = X_train[numerical_attrs]
X_test_num = X_test[numerical_attrs]

In [ ]:
# Scale the numerical attributes
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled = scaler.transform(X_test_num)

In [ ]:
# Create and train the Gaussian Naïve Bayes Classifier
gnb = GaussianNB()
gnb.fit(X_train_num_scaled, y_train)

GaussianNB()

In [ ]:
# Make predictions on the test set
y_pred = gnb.predict(X_test_num_scaled)

In [ ]:
# Generate and print the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.37      0.62      0.46        63
           1       0.82      0.63      0.71       177

    accuracy                           0.62       240
   macro avg       0.60      0.62      0.59       240
weighted avg       0.70      0.62      0.65       240



In [ ]:
# Print the accuracy score
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

Accuracy: 0.6250


## Exercies 3 - SVM Classifier (35 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attributes**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.
 - You **must** show the first five row of your encoded dataset, as well as the shape of your train test split. **No credit** will be given if this step is omitted.

In [ ]:
# Remember to continue the task with your processed data from Exercise 1
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
df = pd.read_csv('processed_garments_worker_productivity_processed.csv')

In [ ]:
# Identify categorical and numerical columns
categorical_attributes = ['day', 'quarter', 'department', 'team']
numerical_attributes = ['no_of_workers', 'no_of_style_change', 'smv', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men']

In [ ]:
# Ceeate a ColumnTransformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop = 'first', sparse_output=False), categorical_attributes)
    ],
    remainder='passthrough'
)

In [ ]:
# Prepare the feature matrix X and target variable y
X = df.drop('satisfied', axis=1)
y = df['satisfied']

In [ ]:
# Fit the preprocessor and transform the data
X_encoded = preprocessor.fit_transform(X)

In [ ]:
# Get the feature names after one-hot encoding
onehot_encoder = preprocessor.named_transformers_['onehot']
encoded_feature_names = onehot_encoder.get_feature_names_out(categorical_attributes).tolist()
feature_names = encoded_feature_names + numerical_attributes

In [ ]:
# Create a new dataframe with encoded features
X_encoded_df = pd.DataFrame(X_encoded, columns=feature_names)

In [ ]:
# Split the date into training and testing sets (80:20 ratio)
X_train, X_test, y_train, y_test = train_test_split(X_encoded_df, y, test_size=0.2, random_state=42)
print("Shape of training set:", X_train.shape)
print("Shape of testing set:", X_test.shape)

Shape of training set: (957, 30)
Shape of testing set: (240, 30)


In [ ]:
# Display the first few rows of the encoded features
print(X_train.head())

      day_Saturday  day_Sunday  day_Thursday  day_Tuesday  day_Wednesday  \
1189           0.0         0.0           0.0          0.0            1.0   
575            0.0         0.0           0.0          0.0            0.0   
76             0.0         0.0           0.0          0.0            0.0   
731            0.0         0.0           1.0          0.0            0.0   
138            0.0         0.0           1.0          0.0            0.0   

      quarter_Quarter2  quarter_Quarter3  quarter_Quarter4  quarter_Quarter5  \
1189               1.0               0.0               0.0               0.0   
575                0.0               0.0               0.0               0.0   
76                 0.0               0.0               0.0               0.0   
731                1.0               0.0               0.0               0.0   
138                1.0               0.0               0.0               0.0   

      department_finishing   ...  team_11  team_12  no_of_work

In [ ]:
# Display the first few rows of the target variable
print(y_train.head())

1189    1
575     0
76      1
731     1
138     1
Name: satisfied, dtype: int64


### Exercise 3.2 - SVM with Different Kernels (15 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [ ]:
# Remember to do this task with your processed data from Exercise 3.1
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

In [ ]:
# Create a pipeline with scaling and SVM for each kernel
linear_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='linear', random_state=42))
])

rbf_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', random_state=42))
])

In [ ]:
# Fit the linear SVM
linear_svm.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm', SVC(kernel='linear', random_state=42))])

In [ ]:
# Fit the RBF SVM
rbf_svm.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svm', SVC(random_state=42))])

In [ ]:
# Make predictions
y_pred_linear = linear_svm.predict(X_test)
y_pred_rbf = rbf_svm.predict(X_test)

In [ ]:
# Generate and print the classification reports
# Linear
print(classification_report(y_test, y_pred_linear))
# RBF
print(classification_report(y_test, y_pred_rbf))

              precision    recall  f1-score   support

           0       0.65      0.35      0.45        63
           1       0.80      0.93      0.86       177

    accuracy                           0.78       240
   macro avg       0.72      0.64      0.66       240
weighted avg       0.76      0.78      0.75       240

              precision    recall  f1-score   support

           0       0.59      0.32      0.41        63
           1       0.79      0.92      0.85       177

    accuracy                           0.76       240
   macro avg       0.69      0.62      0.63       240
weighted avg       0.74      0.76      0.74       240



### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class.
 - Oversample the **training data**.
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br >
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [ ]:
# Remember to do this task with your processed data from Exercise 3.1
from imblearn.over_sampling import SMOTE
from collections import Counter

In [ ]:
# Print the frequency of each original class in the training set
print(Counter(y_train))

Counter({1: 698, 0: 259})


In [ ]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Oversample the scaled training data using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

In [ ]:
# Print the frequency of each class in the oversampled training set
print(Counter(y_train_resampled))

Counter({1: 698, 0: 698})


In [ ]:
# Create SVM models
svm_linear = SVC(kernel='linear', random_state=42)
svm_rbf = SVC(kernel='rbf', random_state=42)

In [ ]:
# Fit the models
svm_linear.fit(X_train_resampled, y_train_resampled)
svm_rbf.fit(X_train_resampled, y_train_resampled)

SVC(random_state=42)

In [ ]:
# Make predictions on the scaled test set
y_pred_linear = svm_linear.predict(X_test_scaled)
y_pred_rbf = svm_rbf.predict(X_test_scaled)

In [ ]:
# Generate and print the classification reports
# Linear
print(classification_report(y_test, y_pred_linear))
# RBF
print(classification_report(y_test, y_pred_rbf))

              precision    recall  f1-score   support

           0       0.44      0.60      0.51        63
           1       0.84      0.72      0.78       177

    accuracy                           0.69       240
   macro avg       0.64      0.66      0.64       240
weighted avg       0.73      0.69      0.71       240

              precision    recall  f1-score   support

           0       0.50      0.54      0.52        63
           1       0.83      0.81      0.82       177

    accuracy                           0.74       240
   macro avg       0.67      0.67      0.67       240
weighted avg       0.74      0.74      0.74       240



## 4) Collaborative Statement (5 points)
#### You must fill this out even if you worked alone to get credit.

It is mandatory to include a Statement of Collaboration in each submission, that follows the guidelines below.
Include the names of everyone involved in the discussions (especially in-person ones), and what was discussed.
All students are required to follow the academic honesty guidelines posted on the course website. For
programming assignments in particular, I encourage students to organize (perhaps using Piazza) to discuss the
task descriptions, requirements, possible bugs in the support code, and the relevant technical content before they
start working on it. However, you should not discuss the specific solutions, and as a guiding principle, you are
not allowed to take anything written or drawn away from these discussions (no photographs of the blackboard,
written notes, referring to Piazza, etc.). Especially after you have started working on the assignment, try to restrict
the discussion to Piazza as much as possible, so that there is no doubt as to the extent of your collaboration.

Even if you did not use any outside resources or collaborate with anyone, please state that explicitly in the space below.

I worked independently on this assignment and did not collaborate with anyone. I followed the
academic honesty guidelines and focused solely on my own analysis and coding practices.